In [52]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import train_test_split
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam
import os
from tensorflow.keras.models import load_model

In [ ]:
# getting cwd with os, use if needed:
# path = os.getcwd()

In [54]:
data_path = './training'
data_dir = os.listdir(data_path)

In [55]:
# removing .DS_Store
data_dir.remove('.DS_Store')

In [56]:
# Importing all images and labels for training:

img_data_list = []
labels = []

for data in data_dir:
    img_list = os.listdir(data_path+'/'+data)
    print('Loading images of', data)
    for img in img_list:
        img_path = data_path +'/'+ data + '/' + img
        try:
            img = image.load_img(img_path, target_size= (28,28))
            x = image.img_to_array(img)
            x = preprocess_input(x)
            img_data_list.append(x)
            labels.append(data)
        except:
            continue

Loading images of vegetarian
Loading images of places
Loading images of non_vegetarian


In [57]:
# Converting images into a numpy array:
img_data = np.array(img_data_list)

In [58]:
np.save('./data/train_sq_data.npy', img_data)

In [59]:
np.save('./data/train_sq_labels.npy', labels)

In [60]:
img_data = np.load('./data/train_sq_data.npy')
labels = np.load('./data/train_sq_labels.npy')

In [61]:
# Use if needed
# num_class = 3
# num_of_samples = img_data.shape[0]

In [62]:
# converting labels into numerical values:

numerical_labels = []
for each in labels:
    if each == 'vegetarian':
        numerical_labels.append(0)
    elif each == 'non_vegetarian':
        numerical_labels.append(1)
    else:
        numerical_labels.append(2)

In [63]:
numerical_labels= np.array(numerical_labels)

# Train Test Split:

In [64]:
x = img_data

In [65]:
y = utils.to_categorical(numerical_labels, num_classes=3)

In [66]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42, shuffle=True)

# Modeling :

In [67]:
# More Complexity!
# Instantiate a CNN
cnn_model_2 = Sequential()

# Add a convolutional layer
cnn_model_2.add(Conv2D(filters=16,             # number of filters
                       kernel_size=(3,3),      # height/width of filter
                       activation='relu',      # activation function 
                       input_shape=(28,28,3))) # shape of input (image)

# Add a pooling layer
cnn_model_2.add(MaxPooling2D(pool_size=(2,2))) # dimensions of region of pooling

# Add another convolutional layer
cnn_model_2.add(Conv2D(64,
                       kernel_size=(3,3),
                       activation='relu'))

# Add another pooling layer
cnn_model_2.add(MaxPooling2D(pool_size=(2,2)))

# We have to remember to flatten to go from the "box" to the vertical line of nodes!
cnn_model_2.add(Flatten())

# Add a densely-connected layer with 64 neurons
cnn_model_2.add(Dense(64, activation='relu'))

# Add a densely-connected layer with 32 neurons
cnn_model_2.add(Dense(32, activation='relu'))

# Add a final layer with 3 neurons
cnn_model_2.add(Dense(3, activation='softmax'))

# Compile model
cnn_model_2.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

# Fit model on training data
history = cnn_model_2.fit(X_train,
                          y_train,
                          batch_size=128,
                          validation_data=(X_test, y_test),
                          epochs=10,
                          verbose=0)

# Predicting Test Data:

In [68]:
data_path = './testing'
data_dir = os.listdir(data_path)

In [69]:
data_dir.remove('.DS_Store')

In [70]:
img_data_list = []
labels = []
image_paths = []

for data in data_dir:
    img_list = os.listdir(data_path+'/'+data)
    print('Loading images of', data)
    for img in img_list:
        img_path = data_path+'/'+data + '/' +img
        try:
            img = image.load_img(img_path, target_size= (28,28))
            x = image.img_to_array(img)
            x = preprocess_input(x)
            img_data_list.append(x)
            labels.append(data)
            image_paths.append(img_path)
        except:
            continue

Loading images of vegetarian
Loading images of places
Loading images of non_vegetarian


In [71]:
img_data = np.array(img_data_list)

In [72]:
np.save('./data/testing_sq_data.npy', img_data)

In [73]:
np.save('./data/testing_sq_labels.npy', labels)

In [74]:
img_data = np.load('./data/testing_sq_data.npy')
labels = np.load('./data/testing_sq_labels.npy')

In [75]:
num_class = 3
num_of_samples = img_data.shape[0]

In [76]:
num_labels = []

for each in labels:
    if each == 'vegetarian':
        num_labels.append(0)
    elif each == 'non_vegetarian':
        num_labels.append(1)
    else:
        num_labels.append(2)

In [77]:
num_labels= np.array(num_labels)

In [78]:
x = img_data

In [79]:
y = utils.to_categorical(num_labels, num_classes=3)

# Predicting on test Data:

In [80]:
cnn_model_2

In [81]:
non_vegetarian_images = []

In [82]:
places_images = []

In [83]:
vegetarian_images = []

In [84]:
for i, im in enumerate(x):
    test_img = np.expand_dims(im, axis=0)
    single_pred = cnn_model_2.predict(test_img,verbose=0)
    if np.argmax(single_pred) == 0:
        vegetarian_images.append(image_paths[i])
    elif np.argmax(single_pred)==1:
        non_vegetarian_images.append(image_paths[i])
    else:
        places_images.append(image_paths[i])

In [85]:
vegetarian_images

["./testing/vegetarian/Little Frank's Pizzeria+9myimage.jpg",
 './testing/vegetarian/Benihana+7myimage.jpg',
 './testing/vegetarian/Indulge Diner+2myimage.jpg',
 './testing/vegetarian/Jade Siam+6myimage.jpg',
 './testing/vegetarian/La Fonda Latina+5myimage.jpg',
 './testing/vegetarian/Indulge Diner+7myimage.jpg',
 './testing/vegetarian/Bacio Pizzeria+7myimage.jpg',
 './testing/vegetarian/Cafe Rio Mexican Grill+6myimage.jpg',
 "./testing/vegetarian/Angelo's Coal Oven Pizzeria+8myimage.jpg",
 './testing/vegetarian/Bella Blu+8myimage.jpg',
 './testing/vegetarian/Baja Fresh+5myimage.jpg',
 './testing/vegetarian/3919555.jpg',
 './testing/vegetarian/Boqueria Soho+4myimage.jpg',
 './testing/vegetarian/Le Fat+9myimage.jpg',
 "./testing/vegetarian/Hugo's Cellar+5myimage.jpg",
 './testing/vegetarian/Cadillac Mexican Kitchen & Tequila Bar+8myimage.jpg',
 './testing/vegetarian/Marea+4myimage.jpg',
 './testing/vegetarian/Cipriani Downtown NYC+9myimage.jpg',
 "./testing/vegetarian/Michael's+5myimage

In [86]:
cnn_model_2.save('cnn_model_2.h5')

new_model = load_model('cnn_model_2.h5')

new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 16)        448       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        9280      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dense_3 (Dense)             (None, 64)               

In [88]:
# More Complexity!
# Instantiate a CNN
cnn_model_3 = Sequential()

# Add a convolutional layer
cnn_model_3.add(Conv2D(filters=64,             # number of filters
                       kernel_size=(3,3),      # height/width of filter
                       activation='relu',      # activation function 
                       input_shape=(28,28,3))) # shape of input (image)

# Add a pooling layer
cnn_model_3.add(MaxPooling2D(pool_size=(2,2))) # dimensions of region of pooling

# Add another convolutional layer
cnn_model_3.add(Conv2D(32,
                       kernel_size=(3,3),
                       activation='relu'))

# Add another pooling layer
cnn_model_3.add(MaxPooling2D(pool_size=(2,2)))

# We have to remember to flatten to go from the "box" to the vertical line of nodes!
cnn_model_3.add(Flatten())

# Add a densely-connected layer with 64 neurons
cnn_model_3.add(Dense(64, activation='relu'))

# Add a densely-connected layer with 32 neurons
cnn_model_3.add(Dense(32, activation='relu'))

# Add a final layer with 3 neurons
cnn_model_3.add(Dense(3, activation='softmax'))

# Compile model
cnn_model_3.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

# Fit model on training data
history = cnn_model_3.fit(X_train,
                          y_train,
                          batch_size=128,
                          validation_data=(X_test, y_test),
                          epochs=10,
                          verbose=1)

Epoch 1/10
21/21 [==============================] - 2s 71ms/step - loss: 4.1669 - accuracy: 0.4775 - val_loss: 1.1887 - val_accuracy: 0.5473
Epoch 2/10
21/21 [==============================] - 1s 66ms/step - loss: 0.9829 - accuracy: 0.5915 - val_loss: 0.9427 - val_accuracy: 0.6171
Epoch 3/10
21/21 [==============================] - 1s 71ms/step - loss: 0.8439 - accuracy: 0.6486 - val_loss: 0.8623 - val_accuracy: 0.6481
Epoch 4/10
21/21 [==============================] - 2s 82ms/step - loss: 0.7099 - accuracy: 0.7149 - val_loss: 0.8454 - val_accuracy: 0.6682
Epoch 5/10
21/21 [==============================] - 2s 89ms/step - loss: 0.6694 - accuracy: 0.7358 - val_loss: 0.7934 - val_accuracy: 0.6899
Epoch 6/10
21/21 [==============================] - 2s 74ms/step - loss: 0.6083 - accuracy: 0.7603 - val_loss: 0.8116 - val_accuracy: 0.6822
Epoch 7/10
21/21 [==============================] - 1s 70ms/step - loss: 0.5862 - accuracy: 0.7611 - val_loss: 0.7614 - val_accuracy: 0.6930
Epoch 8/10
21

In [ ]:
200, 200